In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/celestial/input/data_raw.pkl
/kaggle/input/celestial/input/data_cleaned.pkl


In [2]:
import pandas as pd
import numpy as np

import dill as pickle

In [3]:
with open("/kaggle/input/celestial/input/data_cleaned.pkl", "rb") as f:
    data = pickle.load(f)

In [4]:
train = data["train"]
test = data["test"]

In [5]:
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import log_loss,roc_auc_score
from catboost import CatBoostClassifier, Pool
pd.set_option("display.precision", 15)
np.set_printoptions(suppress=True)

In [25]:
N_SPLITS = 10
SEED = 42
YCLASS = 19
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
target = "type"
oof = pd.DataFrame(columns = np.arange(YCLASS),index = np.arange(len(train)))
predictions = np.zeros((len(test), YCLASS))

features = [col for col in test.columns if col not in [target, 'id']]
X_test = test[features].values

for fold, (trn_idx, val_idx) in enumerate(skf.split(train, train[target])):
    X_train, y_train = train.iloc[trn_idx][features], train.iloc[trn_idx][target]
    X_valid, y_valid = train.iloc[val_idx][features], train.iloc[val_idx][target]
    trn_data = Pool(X_train, label=y_train)
    val_data = Pool(X_valid, label=y_valid)
    evals_result = {}
    params = {"iterations" : 1500}
    
    model = CatBoostClassifier(**params,
                            loss_function="MultiClass",
                           task_type="GPU")

    fit_model = model.fit(trn_data,
                          eval_set=val_data,
                          use_best_model=True,
                          verbose=500,
                          #plot=True
                         )
    p_valid = fit_model.predict_proba(X_valid[features].values)
    yp = fit_model.predict_proba(X_test)

    p_valid_df = pd.DataFrame(p_valid, columns = np.arange(YCLASS), index = X_valid.index)
    logloss = log_loss(y_valid, p_valid)

    oof = oof.combine_first(p_valid_df)

    predictions += yp / N_SPLITS

Learning rate set to 0.148819
0:	learn: 1.6950077	test: 1.6988023	best: 1.6988023 (0)	total: 22.5ms	remaining: 33.7s
500:	learn: 0.3566291	test: 0.3886285	best: 0.3886285 (500)	total: 8.89s	remaining: 17.7s
1000:	learn: 0.3154598	test: 0.3752576	best: 0.3752576 (1000)	total: 19.3s	remaining: 9.64s
1499:	learn: 0.2866894	test: 0.3708401	best: 0.3708327 (1482)	total: 28.2s	remaining: 0us
bestTest = 0.3708327393
bestIteration = 1482
Shrink model to first 1483 iterations.
Learning rate set to 0.14882
0:	learn: 1.7090449	test: 1.7016009	best: 1.7016009 (0)	total: 19.9ms	remaining: 29.8s
500:	learn: 0.3566129	test: 0.3852388	best: 0.3852388 (500)	total: 8.85s	remaining: 17.6s
1000:	learn: 0.3151649	test: 0.3705102	best: 0.3705102 (1000)	total: 17.9s	remaining: 8.9s
1499:	learn: 0.2863693	test: 0.3654120	best: 0.3654120 (1499)	total: 26.5s	remaining: 0us
bestTest = 0.365412045
bestIteration = 1499
Learning rate set to 0.14882
0:	learn: 1.6939098	test: 1.7018240	best: 1.7018240 (0)	total: 18.8

In [66]:
data["keys"]

Index(['QSO', 'STAR_RED_DWARF', 'SERENDIPITY_BLUE', 'STAR_BHB',
       'STAR_CATY_VAR', 'SERENDIPITY_DISTANT', 'GALAXY', 'SPECTROPHOTO_STD',
       'REDDEN_STD', 'ROSAT_D', 'STAR_WHITE_DWARF', 'SERENDIPITY_RED',
       'STAR_CARBON', 'SERENDIPITY_FIRST', 'STAR_BROWN_DWARF',
       'STAR_SUB_DWARF', 'SKY', 'SERENDIPITY_MANUAL', 'STAR_PN'],
      dtype='object')

In [73]:
col = ["id","STAR_WHITE_DWARF","STAR_CATY_VAR","STAR_BROWN_DWARF","SERENDIPITY_RED",
"REDDEN_STD","STAR_BHB","GALAXY", "SERENDIPITY_DISTANT","QSO","SKY","STAR_RED_DWARF","ROSAT_D",
"STAR_PN","SERENDIPITY_FIRST","STAR_CARBON","SPECTROPHOTO_STD","STAR_SUB_DWARF","SERENDIPITY_MANUAL",
 "SERENDIPITY_BLUE"]


In [75]:
yp_submission = pd.DataFrame(predictions, columns = data["keys"]) 
yp_submission["id"] =  test.id
yp_submission = yp_submission[col]

In [69]:
params = {"iterations" : 1500}

In [70]:
filename = "submission1.csv"
training_loss = fit_model.get_best_score()["learn"]["MultiClass"]
validation_loss = fit_model.get_best_score()["validation"]["MultiClass"]
public_score = 0
description = params

In [71]:
result = pd.DataFrame(data = [[filename, training_loss, validation_loss, public_score, description]],
             columns = ["filename", "training_loss", "validation_loss", "public_score", "description"])

In [76]:
yp_submission.to_csv(filename,index = False)
result.to_csv("result.csv",index=False)